# Implementation Practice.1
## Forward and Backward Propagation with $y = \theta x$

***
### Dataset Generation
NumPy를 이용하여 다음과 같은 x_data, y_data를 만드시오.

| Data Index | Study Hour | Math Score |
|:-:|:-:|:-:|
|0|1|1|
|1|2|2|
|2|3|3|
|3|4|4|
|4|5|5|


In [1]:
import numpy as np
import matplotlib as plt

py_array = [1,2,3,4,5]
x_data = np.array(py_array)
y_data = np.array(py_array)
py_array.append(6)

print(x_data, y_data)

[1 2 3 4 5] [1 2 3 4 5]


**Expected Output:**
    

[1 2 3 4 5] [1 2 3 4 5]


***
### Node Implementation
다음은 plus node의 예제이다.

우리의 model이 $y = \theta x$였을 때, $(x^{(1)},y^{(1)}) $에 대하여 loss를 구하는 과정은 다음과 같다.

Step1: $ z_{1}^{(1)} = \theta x^{(1)} $

Step2: $ z_{2}^{(1)} = y^{(1)} - z_{1}^{(1)} $

Step3: $ L^{(1)} = (z_{2}^{(1)})^{2} $


위의 과정을 forward propagation이라하며, 차례대로 곱셈, 뺄셈, 제곱이 이루어진다.

그러면 위의 과정 외에 forward-propagation 및 backpropagation을 연습하기 위하여 덧셈, 뺄셈, 곱셈, 제곱에 대한 node를 만들어 각각 forward, backward 기능을 추가해보자. <br><br>

forward() method는 input들을 더하여 다음 node로 보내주는 역할을 하며

backward() method는 output 쪽에서 propagated된 loss에 대한 gradient를 받아 자신의 gradient를 곱하여 input 쪽으로 넘겨준다. 즉

$ z = x + y$에 대하여 $\frac{\partial z}{\partial x} = 1$, $\frac{\partial z}{\partial y} = 1$ 이므로 Chain rule에 의해 Output node 쪽에서 넘어온 dL과 1, 1을 곱해 앞단으로 $1*dL$, $1*dL$을 넘겨주는 코드이다.

In [2]:
import numpy as np

class plus_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x + y
        return self.z
    def backward(self, dL):
        return 1*dL, 1*dL
class minus_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x-y
        return self.z
    def backward(self, dL):
        return 1*dL, -1*dL

class mul_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x*y
        return self.z
    def backward(self, dL):
        return self.y*dL, self.x*dL
        
class square_node():
    def __init__(self):
        self.x, self.z = None, None
    def forward(self, x):
        self.x, self.z = x, np.square(x)
        return self.z
    def backward(self, dL):
        return 2*self.x*dL

test_plus = plus_node()
print("plus node(forward):", test_plus.forward(3, 4))
print("plus node(backward):", test_plus.backward(5), '\n')

test_minus = minus_node()
print("minus node(forward):", test_minus.forward(3, 4))
print("minus node(backward):", test_minus.backward(5), '\n')

test_mul = mul_node()
print("multiplication node(forward):", test_mul.forward(3, 4))
print("multiplication node(backward):", test_mul.backward(5), '\n')

test_square = square_node()
print("square node(forward):", test_square.forward(3))
print("square node(backward):", test_square.backward(5), '\n')

plus node(forward): 7
plus node(backward): (5, 5) 

minus node(forward): -1
minus node(backward): (5, -5) 

multiplication node(forward): 12
multiplication node(backward): (20, 15) 

square node(forward): 9
square node(backward): 30 



**Expected Output:**

plus node(forward): 7 <br>
plus node(backward): (5, 5) <br><br>

minus node(forward): -1 <br>
minus node(backward): (5, -5) <br><br>

multiplication node(forward): 12 <br>
multiplication node(backward): (20, 15) <br><br> 

square node(forward): 9 <br>
square node(backward): 90 


***
### Forward & Backward Propagation and Parameter Update
위의 node들을 이용하여 전체 model을 만들고, forward propagation, backward propagation을 구현하여 $\theta$를 update하는 코드를 작성하시오

In [3]:
import numpy as np
import matplotlib.pyplot as plt

py_array = [1,2,3,4,5]
x_data = np.array(py_array)
y_data = np.array(py_array)

theta = 0
lr = 0.001
epochs = 10

##### Your Code(Model Implmenataion) #####
class plus_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x + y
        return self.z
    def backward(self, dL):
        return 1*dL, 1*dL
    
class minus_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x-y
        return self.z
    def backward(self, dL):
        return 1*dL, -1*dL

class mul_node():
    def __init__(self):
        self.x, self.y, self.z = None, None, None
    def forward(self, x, y):
        self.x, self.y, self.z = x, y, x*y
        return self.z
    def backward(self, dL):
        return self.y*dL, self.x*dL
        
class square_node():
    def __init__(self):
        self.x, self.z = None, None
    def forward(self, x):
        self.x, self.z = x, np.square(x)
        return self.z
    def backward(self, dL):
        return 2*self.x*dL
    
plus=plus_node()
minus = minus_node()
mul = mul_node()
square = square_node()

##### Your Code(Model Implmenataion) #####
loss_list = []
theta_list = []
for i in range(epochs):
    z1 = mul.forward(x_data, theta)
    z2 = minus.forward(y_data,z1)
    loss_list.append(square.forward(z2))
    ##### Your Code(Forward Propagation) #####
    
    ##### Your Code(Backward Propagation) #####
    dz2 = square.backward(1)
    dy,dz1 = minus.backward(dz2)
    dx,dtheta = mul.backward(dz1)
    theta = theta - lr*dtheta
    theta_list.append(theta)
    ##### Your Code(Backward Propagation) #####
        
fig, ax = plt.subplots(figsize = (10,10))
ax.plot(loss_list)
ax.set_title("loss")

fig, ax = plt.subplots(figsize = (10,10))
ax.plot(theta_list)
ax.set_title("theta")

# =============================================================================
# Q1) 위의 결과에서 Loss와 theta 모두 부드러운 곡선의 형태가 나타나지 않는 이유를 분석하시오
#     Loss의 경우 : (y-theta*x)^2 꼴로 크기가 2차 방정식의 형태로 상승한다.
#                   그 다음 epoch에서 x[0]과 비교하므로 loss값이 작은 값 부터 다시 2차 방정식의 형태로 상승한다.
#     theta의 경우 : loss값의 기울기에 비례하여 크기가 변하므로 epoch의 시작점 마다 낮은 기울기를 가지는 형태가 된다.
# =============================================================================


Text(0.5, 1.0, 'theta')

**Expected Output:**

<img src="./images/1_1_image1.png" width = 400>

<img src="./images/1_1_image2.png" width = 400>

**Q1) 위의 결과에서 Loss와 theta 모두 부드러운 곡선의 형태가 나타나지 않는 이유를 분석하시오** <br>



